In [2]:
# !pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/0e/59/aae37fa93e2d4292c3148efcc3066c8ecfe5cfaa72bf8c0b1a5614622cf7/selenium-4.15.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/39/46/620fbe56f41fa3ccdda2136d947fb9bacce3d1eb163f057f0262a0ddf5e0/trio-0.23.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/58.3 kB ? eta -:--:--
     ---------------------------------------- 58.3/58.3 kB 1.5 MB/s eta 0:00:00
   ---------------

In [3]:
# !pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b1/51/b5c11cf739ac4eecde611794a0ec9df420d0239d51e73bc19eb44f02b48b/webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata


In [4]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re

In [7]:
# Create driver
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()


# Get the page url 
page_url = "https://gilmoregirls.fandom.com/wiki/Category:Characters"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

In [10]:
# Accept the cookies 
# driver.find_element(By.XPATH, "//div[text()='AKCEPTUJĘ']").click()
driver.find_element(By.CLASS_NAME, "_2O--J403t2VqCuF8XJAZLK").click()

In [47]:
# Create a list of characters
character_element = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')

In [48]:
# # Append characters to the list 
# characters = []
# for value in character_element[8:]:
#     characters_list = value.text
#     characters.append({'full name' : characters_list})

In [91]:
characters_links = [item.get_attribute("href") for item in driver.find_elements(By.CLASS_NAME, 'category-page__member-link')] 
characters_info = [] 

# # Scrape scripts from links 
for item_link in characters_links[8:]:
    driver.get(item_link) # Click the link  
    time.sleep(3)
    name = driver.find_element(By.CLASS_NAME, 'mw-page-title-main').text
    episodes = [element.text for element in driver.find_elements(By.XPATH, "//*[contains(text(), 'Episodes')]/following-sibling::div")]
    actor = [element.text for element in driver.find_elements(By.XPATH, "//*[contains(text(), 'Portrayed by')]/following-sibling::div")]
    if len(actor) == 0: 
        actor = '' 
    elif len(episodes) == 0:
        episodes = '' 
    else: 
        actor = actor[0]
        episodes = episodes[0]
    characters_info.append({'full name': name, 'portrayed by': actor, 'num_episodes': episodes})


In [92]:
characters_info

[{'full name': 'A.K.',
  'portrayed by': ['Adam Hendershott'],
  'num_episodes': ''},
 {'full name': 'Al', 'portrayed by': '', 'num_episodes': []},
 {'full name': 'Alex Lesman',
  'portrayed by': 'Billy Burke',
  'num_episodes': '3'},
 {'full name': 'Allen Prescott',
  'portrayed by': ['Richard Fancy'],
  'num_episodes': ''},
 {'full name': 'Andrew',
  'portrayed by': 'Mike Gandolfi',
  'num_episodes': '22'},
 {'full name': 'Anna Nardini',
  'portrayed by': 'Sherilyn Fenn',
  'num_episodes': '8'},
 {'full name': 'April Nardini',
  'portrayed by': 'Vanessa Marano',
  'num_episodes': '14'},
 {'full name': 'Reverend Archie Skinner',
  'portrayed by': 'Jim Jansen',
  'num_episodes': '7'},
 {'full name': 'Asher Fleming',
  'portrayed by': 'Michael York',
  'num_episodes': '4'},
 {'full name': 'Babette Dell',
  'portrayed by': 'Sally Struthers',
  'num_episodes': '54'},
 {'full name': 'Beau Belleville',
  'portrayed by': 'Nick Offerman',
  'num_episodes': '2'},
 {'full name': 'Bill', 'portra

In [124]:
# Create a dataframe 
characters_df = pd.DataFrame(characters_info)

# Separate name from the full name 
characters_df['name'] = characters_df['full name'].apply(lambda x: x if 
                                                   x.startswith(("Ms.", "Miss", "Mrs.", "Mr")) else x.split()[0]) 

# Correct spelling mistakes 
characters_df.at[24, 'name'] = 'Chris'
characters_df.at[62, 'name'] = 'Digger'
characters_df.at[84, 'name'] = 'Trix'
characters_df.at[152, 'name'] = 'Chui'



In [126]:
characters_df['portrayed by'] = characters_df['portrayed by'].apply(lambda x: ''.join(x))
characters_df['num_episodes'] = characters_df['num_episodes'].apply(lambda x: ''.join(x))

In [127]:
characters_df[:30]

,full name,portrayed by,num_episodes,name
0,A.K.,Adam Hendershott,,A.K.
1,Al,,,Al
2,Alex Lesman,Billy Burke,3,Alex
3,Allen Prescott,Richard Fancy,,Allen
4,Andrew,Mike Gandolfi,22,Andrew
5,Anna Nardini,Sherilyn Fenn,8,Anna
6,April Nardini,Vanessa Marano,14,April
7,Reverend Archie Skinner,Jim Jansen,7,Reverend
8,Asher Fleming,Michael York,4,Asher
9,Babette Dell,Sally Struthers,54,Babette


In [129]:
characters_df[(characters_df['portrayed by'] == '') | (characters_df['num_episodes'] == '')]

,full name,portrayed by,num_episodes,name
0,A.K.,Adam Hendershott,,A.K.
1,Al,,,Al
3,Allen Prescott,Richard Fancy,,Allen
14,Brennon Lewis,Greg Cipes,,Brennon
18,Carol Stiles,Catherine McGoohan,,Carol
20,Charles Gilmore,,Episodes,Charles
21,Charlie Eichner,,,Charlie
22,Christine,Tessa Ludwick,,Christine
24,Cissy,Kara Holden,,Chris
28,Davey Belleville,,,Davey


In [128]:
# save to csv
characters_df.to_csv('characters_info.csv', sep='\t', index=False)

NameError: name 'characters_df' is not defined